#### Modelagem

$$
min \space f(x) = \sum_{i=1}^{n}\sum_{j=1}^{n}D_{ij}w_{ij}
$$

##### Variáveis

* $w_{ij} \in \{0,1\}$ -> igual a 1 se a localidade $i$ é atendida pelo polo $j$
* $z_j \in \{0,1\}$ -> igual a 1 se a localidade $j$ é um polo industrial

##### Restrições

* De todas as $n$ localidades, terão $p$ que serão polos:
$$
\sum_{j=1}^{n}z_j = p
$$

* Cada uma das $n$ seja atendida por exatamente uma das $p$ localidades:
$$
\sum_{j=1}^{n}w_{ij} = 1,\space ∀i \in \{1,2,..,n\}
$$

* Uma localidade só pode atender se for um polo
$$
w_{ij} \leq z_j, \space \forall i, j \in \{1,2,...,n\}
$$

* A soma das demandas atendidas por uma localidade determinada não exceda a sua capacidade.
$$
\sum_{i=1}^{n}d_iw_{ij}\leq c_jz_j ,\space \forall j \in \{1,2,...,n\}
$$

* Cálculo das distâncias euclidianas

$$
D_{ij} = \sqrt{(x_i-x_j)^2 + (y_i-y_j)^2}
$$

In [9]:
import pandas as pd
import numpy as np
from gurobipy import GRB, Model
import math

data = pd.read_csv("p1.dat", header=None)

localidades = []
for index, row in data.iterrows():
    # Split the string by spaces
    numbers_str = row[0].split()

    # Convert the string numbers to integers
    numbers_int = [int(num) for num in numbers_str]

    if index==1:
        total_localidades = numbers_int[0]
        p_localidades = numbers_int[1]
        capacidade_cada_p_localidade = numbers_int[2]
    if index > 1:
        localidades.append(numbers_int)

modelo = Model("sucroalcooleiro")

z = {}
w = {}
for i in range(total_localidades):
    z[i] = modelo.addVar(vtype=GRB.BINARY, name=f"z_{i}")
    w[i] = {}
    for j in range(total_localidades):
        w[i][j] = modelo.addVar(vtype=GRB.BINARY, name=f"w_{i}_{j}")

modelo.setObjective(sum(sum( w[i][j]*math.sqrt((localidades[i][1] - localidades[j][1])**2+(localidades[i][2] - localidades[j][2])**2)for j in range(total_localidades)) for i in range(total_localidades)), GRB.MINIMIZE)


modelo.addConstr(sum(z[i] for i in range(total_localidades)) == p_localidades, "total_polos")

for i in range(total_localidades):
  for j in range(total_localidades):
    modelo.addConstr(w[i][j] <= z[j], "somente_polo_atende")

for i in range(total_localidades):
    modelo.addConstr(sum(w[i][j] for j in range(total_localidades)) == 1, f"localidade_{i} atendida por 1 polo")

for j in range(total_localidades):
    modelo.addConstr(sum(localidades[i][3] * w[i][j] for i in range(total_localidades)) <= capacidade_cada_p_localidade*z[j], f"capacidade_polo_{j}")

modelo.optimize()



Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-5200U CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 2601 rows, 2550 columns and 10100 nonzeros
Model fingerprint: 0x85cbee2f
Variable types: 0 continuous, 2550 integer (2550 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Found heuristic solution: objective 2812.4619692
Presolve time: 0.07s
Presolved: 2601 rows, 2550 columns, 10100 nonzeros
Variable types: 0 continuous, 2550 integer (2550 binary)

Root relaxation: objective 7.150401e+02, 692 iterations, 0.07 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  715.04007 

In [11]:
if(modelo.status == GRB.OPTIMAL):
    print(f"Valor otimo: {modelo.ObjVal}")
    print("Polos industriais")
    for i in range(total_localidades):
        if(z[i].x == 1):
            print(f"Localidade {i+1} eh polo industrial e atende")
            for j in range(total_localidades):
                if(w[j][i].x == 1):
                    print(f"\tLocalidade {j+1}")

Valor otimo: 728.2620477765407
Polos industriais
Localidade 12 eh polo industrial e atende
	Localidade 2
	Localidade 6
	Localidade 8
	Localidade 9
	Localidade 12
	Localidade 20
	Localidade 25
	Localidade 35
	Localidade 40
	Localidade 43
Localidade 17 eh polo industrial e atende
	Localidade 3
	Localidade 7
	Localidade 10
	Localidade 13
	Localidade 17
	Localidade 23
	Localidade 30
	Localidade 38
	Localidade 42
	Localidade 45
	Localidade 46
	Localidade 49
Localidade 19 eh polo industrial e atende
	Localidade 4
	Localidade 5
	Localidade 19
	Localidade 22
	Localidade 24
	Localidade 27
	Localidade 28
	Localidade 29
	Localidade 31
	Localidade 37
	Localidade 47
Localidade 21 eh polo industrial e atende
	Localidade 1
	Localidade 11
	Localidade 14
	Localidade 15
	Localidade 18
	Localidade 21
	Localidade 32
	Localidade 36
	Localidade 39
	Localidade 41
	Localidade 44
	Localidade 50
Localidade 48 eh polo industrial e atende
	Localidade 16
	Localidade 26
	Localidade 33
	Localidade 34
	Localidade 48
